In [1]:
import requests
import pandas as pd

BASE_URL = "https://gamma-api.polymarket.com/markets"
# params = {
#     "limit": 100000,
#     "active": "true"
# }

# response = requests.get(BASE_URL, params=params)
# data = response.json()
# markets = data

all_markets = []
offset = 0
while True:
    params = {"limit": 100, "offset": offset}
    r = requests.get(BASE_URL, params=params)
    r.raise_for_status()
    batch = r.json()
    if not batch:
        break
    all_markets.extend(batch)
    offset += 100

# Filter ETH markets only
eth_markets = []
keywords = ["eth", "ethereum", "eth", "ether", "ethusd", "Ethereum"]

for m in all_markets:
    question = m.get("question", "").lower()
    if any(k in question for k in keywords):
        eth_markets.append({
            "id": m.get("id"),
            "question": m.get("question"),
            "slug": m.get("slug"),
            "start_date": m.get("start_date"),
            "end_date": m.get("end_date"),
            "volume": m.get("volume_num"),
            "liquidity": m.get("liquidity_num"),
            "active": m.get("active"),
            "closed": m.get("closed")
        })

# Save filtered ETH markets
df = pd.DataFrame(eth_markets)
df.to_csv("eth_betting_markets.csv", index=False)
print(f"✅ Saved {len(df)} ETH markets.")
df.head(10)


HTTPError: 429 Client Error: Too Many Requests for url: https://gamma-api.polymarket.com/markets?limit=100&offset=4400

In [4]:
import requests
import pandas as pd
import time  # Add this import
import random  # Add this for randomizing sleep time

BASE_URL = "https://gamma-api.polymarket.com/markets"

all_markets = []
offset = 0
max_retries = 5  # Set a maximum number of retries
max_offset = 10000  # Set a reasonable maximum to avoid infinite loops

while offset < max_offset:
    params = {"limit": 100, "offset": offset}
    
    # Add retry logic
    for retry in range(max_retries):
        try:
            # Add a delay between requests to avoid rate limiting
            time.sleep(random.uniform(1.0, 2.0))
            
            r = requests.get(BASE_URL, params=params)
            r.raise_for_status()
            batch = r.json()
            
            if not batch:
                max_offset = 0  # This will exit the outer loop
                break
                
            all_markets.extend(batch)
            offset += 100
            print(f"Successfully fetched batch at offset {offset-100}, got {len(batch)} markets")
            break  # Success, exit retry loop
            
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 429:
                # If rate limited, wait longer before retrying
                wait_time = 5 + random.uniform(5.0, 15.0)
                print(f"Rate limited. Waiting {wait_time:.2f} seconds before retry {retry+1}/{max_retries}")
                time.sleep(wait_time)
                continue
            elif retry == max_retries - 1:
                print(f"Failed after {max_retries} retries: {e}")
                max_offset = 0  # Exit outer loop
                break
            else:
                print(f"Error: {e}. Retrying ({retry+1}/{max_retries})...")
                time.sleep(2)
        except Exception as e:
            print(f"Unexpected error: {e}")
            if retry == max_retries - 1:
                max_offset = 0  # Exit outer loop
                break
            time.sleep(2)

# Filter ETH markets only
eth_markets = []
keywords = ["eth", "ethereum", "eth", "ether", "ethusd", "Ethereum"]

for m in all_markets:
    question = m.get("question", "").lower()
    if any(k in question for k in keywords):
        eth_markets.append({
            "id": m.get("id"),
            "question": m.get("question"),
            "slug": m.get("slug"),
            "start_date": m.get("start_date"),
            "end_date": m.get("end_date"),
            "volume": m.get("volume_num"),
            "liquidity": m.get("liquidity_num"),
            "active": m.get("active"),
            "closed": m.get("closed")
        })

# Save filtered ETH markets
df = pd.DataFrame(eth_markets)
df.to_csv("eth_betting_markets.csv", index=False)
print(f"✅ Saved {len(df)} ETH markets.")
df.head(10)

Successfully fetched batch at offset 0, got 100 markets
Successfully fetched batch at offset 100, got 100 markets
Successfully fetched batch at offset 200, got 100 markets
Successfully fetched batch at offset 300, got 100 markets
Successfully fetched batch at offset 400, got 100 markets
Successfully fetched batch at offset 500, got 100 markets
Successfully fetched batch at offset 600, got 100 markets
Successfully fetched batch at offset 700, got 100 markets
Successfully fetched batch at offset 800, got 100 markets
Successfully fetched batch at offset 900, got 100 markets
Successfully fetched batch at offset 1000, got 100 markets
Successfully fetched batch at offset 1100, got 100 markets
Successfully fetched batch at offset 1200, got 100 markets
Successfully fetched batch at offset 1300, got 100 markets
Successfully fetched batch at offset 1400, got 100 markets
Successfully fetched batch at offset 1500, got 100 markets
Successfully fetched batch at offset 1600, got 100 markets
Successfu

,id,question,slug,start_date,end_date,volume,liquidity,active,closed
0,75,Will the Ethereum 2.0 Genesis Event happen suc...,will-the-ethereum-20-genesis-event-happen-succ...,None,None,None,None,True,True
1,8938,"Will ETH be above $1,500 on January 27th?",will-eth-be-above-1-500-on-january-27th,None,None,None,None,True,True
2,61328,"Will ETH be above $1500 on February 3rd, 2021?",will-eth-be-above-1500-on-february-3rd-2021,None,None,None,None,True,True
3,71914,"Will ETH be above $2000 on March 1st, 2021?",will-eth-be-above-2000-on-march-1st-2021,None,None,None,None,True,True
4,79704,Will the average Ethereum gas price be below 1...,will-the-average-ethereum-gas-price-be-below-1...,None,None,None,None,True,True
5,98033,"Will ETH be above $2000 on April 1st, 2021?",will-eth-be-above-2000-on-april-1st-2021,None,None,None,None,True,True
6,98685,"Will ETH be above $1500 on March 7th, 2021?",will-eth-be-above-1500-on-march-7th-2021,None,None,None,None,True,True
7,101793,"Will ETH be above $1750 on March 22nd, 2021?",will-eth-be-above-1750-on-march-22nd-2021-1,None,None,None,None,True,True
8,101817,Will the average Ethereum gas price be below 1...,will-the-average-ethereum-gas-price-be-below-1...,None,None,None,None,True,True
9,107931,Will the average Ethereum gas price be below 1...,will-the-average-ethereum-gas-price-be-below-1...,None,None,None,None,True,True


In [34]:
def get_market_outcomes(market_id):
    url = f"https://gamma-api.polymarket.com/markets/{market_id}"
    r = requests.get(url)
    if r.status_code != 200:
        return None
    market = r.json()
    outcomes = market.get("outcomes", [])
    return outcomes

# Example: get outcome prices for one market
outcomes = get_market_outcomes("101793")  # replace with real ETH market ID
print(outcomes)


["Yes", "No"]


In [ ]:
import re

def extract_price_threshold(question):
    match = re.search(r"\$\s?([\d,]+)", question)
    if match:
        price_str = match.group(1).replace(",", "")
        return float(price_str)
    return None

question = "Will ETH be above $2000 on April 1st, 2021?	"
print(extract_price_threshold(question))

2000.0


In [19]:
# Save to CSV
df.to_csv("polymarket_crypto_markets.csv", index=False)